In [ ]:
!wget https://s3.amazonaws.com/video.udacity-data.com/topher/2018/October/5bbe6499_text8/text8.zip

In [ ]:
!unzip /content/text8.zip

In [ ]:
with open('/content/text8') as f:
    text =  f.read()

print(text[:100])    

In [ ]:
# nltk
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer

# Word2vec
import gensim

# Utility
import re
import numpy as np
import os
from collections import Counter
import logging
import time
import pickle
import itertools


In [ ]:
nltk.download('stopwords')


In [ ]:
# WORD2VEC 
W2V_SIZE = 300
W2V_WINDOW = 10
W2V_EPOCH = 128
W2V_MIN_COUNT = 10
TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

# EXPORT
WORD2VEC_MODEL = "model.w2v"

In [ ]:
stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")

In [ ]:
def preprocess(text, stem=False):
    # Remove link,user and special characters
    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)


text = preprocess(text)

In [ ]:
text[:100]

In [ ]:
words = text.split()

In [ ]:
documents = [_text.split() for _text in text.split()]


In [ ]:
documents[:30]

In [ ]:
word_counts = Counter(words)
# sorting the words from most to least frequent in text occurrence
sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
# create int_to_vocab dictionaries
int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}


In [ ]:
int_to_vocab

In [ ]:
w2v_model = gensim.models.word2vec.Word2Vec(size=W2V_SIZE, 
                                            window=W2V_WINDOW, 
                                            min_count=W2V_MIN_COUNT, 
                                            workers=8)

In [ ]:
w2v_model.build_vocab(documents)


In [ ]:

vocab_size = len(words)
print("Vocab size", vocab_size)

In [ ]:
w2v_model.train(documents, total_examples=len(documents), epochs=50)


In [ ]:
w2v_model.most_similar("seven")


In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [ ]:
w2v_model.save('w2v_model.wv')

In [ ]:
X = w2v_model[w2v_model.wv.vocab]
tsne = TSNE(n_components=2)
embed_tsne = tsne.fit_transform(X[:1000,:])
fig, ax = plt.subplots(figsize=(16, 16))
for idx in range(viz_words):
    plt.scatter(*embed_tsne[idx, :], color='steelblue')
    plt.annotate(int_to_vocab[idx], (embed_tsne[idx, 0], embed_tsne[idx, 1]), alpha=0.7)